In [2]:
import win32gui
import win32con
import time
import automator
import importlib
importlib.reload(automator)

from ppadb.client import Client as AdbClient
#first one=62001
#second one=62025
my_client = AdbClient()
print(my_client.devices())
my_device = my_client.devices()[0]

hwnd_hyuntamansei = win32gui.FindWindow(None, "hyuntamansei")
hwnd_jchoi82kor = win32gui.FindWindow(None, "jchoi82kor_facebook")
print(hwnd_hyuntamansei, hwnd_jchoi82kor)

0 656032


In [6]:
print(my_automator.locate("select_chapter"))
my_device.input_tap(200,200)

(252, 377)


''

In [4]:
# my_automator = automator.Automator(hwnd_jchoi82kor, './nox_1280_720_images/')
my_automator = automator.Automator(hwnd_jchoi82kor, './nox_images/')
# nox_hyuntamansei = automator.Automator(hwnd_hyuntamansei, './bluestack_images/')
my_automator.confidence=0.8
my_automator.load_conf('nox')
# my_automator.load_conf('nox_1280_720')
my_automator.read_coor('./coor.ini')
my_automator.read_coor()
my_automator.connect_click_method(my_device.input_tap)

for cnt in range(300):
    #퀘스트 자동 진행
    print("selecting chapter")
    # while my_automator.locate('select_chapter') == None:
    #     my_automator.locate_and_click('story')
    #     time.sleep(1)
    my_automator.locate_and_click('select_chapter', xy='top_quest', trial_number=5)
    print("selecting acompany")
    # while my_automator.locate('acompany', trial_number=5):
    #     my_automator.locate_and_click('acompany')
    #     my_automator.locate_and_click('sephiroth')
    #     time.sleep(7)
    # my_automator.locate_and_click('acom_sephiroth')
    print("clicking sortie")
    my_automator.wait_and_click('sortie')
    print("after clicking sortie")
    time.sleep(15)
    ##skip 하기 누르기
    while my_automator.locate('auto') == None:
        my_automator.locate_and_click('sortie')
        print("before battle, story skip")
        my_automator.click('story_skip1')
        time.sleep(2)
        my_automator.click('story_skip2')
        time.sleep(5)
    print("in battle stage")
    my_automator.locate('next', trial_number=600)
    print("Mission complete")
    time.sleep(10)
    while my_automator.locate_and_click('end_of_quest') == None:
        print("Until end_of_quest")
        my_automator.locate_and_click('next')
        # my_automator.locate_and_click('x')
        my_automator.locate_and_click('next')
        # my_automator.locate_and_click('x')
        my_automator.locate_and_click('later')
        my_automator.locate_and_click('close')
        my_automator.locate_and_click('no_evaluate')
    time.sleep(2)
    my_automator.locate_and_click('end_of_quest')
    print("The quest ended")
    time.sleep(10)
    count = 0
    while my_automator.locate('select_chapter', trial_number=2) == None:
        print(f"after batlle, until 'select chapter', repeating, ... story skip, count={count}")
        my_automator.locate_and_click('ok')
        my_automator.locate_and_click('later')
        my_automator.locate_and_click('close')
        if not my_automator.locate('select_chapter'):
            my_automator.click('story_skip1')
            time.sleep(1)
        if not my_automator.locate('select_chapter'):
            my_automator.click('story_skip3')
        count += 1
        my_automator.locate_and_click('story')
    print(f"Completed: {cnt+1} times")

selecting chapter
selecting acompany
clicking sortie
after clicking sortie
in battle stage
Mission complete
Until end_of_quest
Until end_of_quest
Until end_of_quest
Until end_of_quest
Until end_of_quest
Until end_of_quest
Until end_of_quest
Until end_of_quest
Until end_of_quest
Until end_of_quest
Until end_of_quest
Until end_of_quest
Until end_of_quest


KeyboardInterrupt: 

In [15]:
nox_hyuntamansei = automator.Automator(hwnd_hyuntamansei, './nox_images/')
# nox_hyuntamansei = automator.Automator(hwnd_hyuntamansei, './bluestack_images/')
my_automator.confidence=0.95
my_automator.load_conf('nox')
my_automator.read_coor('./coor.ini')
my_automator.connect_click_method(my_device.input_tap)

# my_automator.locate('sortie')
my_automator.locate_and_click('sortie')

(-2000, 213)
(1905, 1107)
Locating: Box(left=1566, top=984, width=102, height=57), center:(1617, 1012)
Saving sc.png
clicking as dev xy: (1617, 985)


(1617, 985)

(0, 0, 1905, 1107)
(-2000, 213, -95, 1320)


In [22]:
my_automator.read_coor()
my_automator.click('story_skip1')
time.sleep(1)
my_automator.click('story_skip2')

clicking as dev xy: (1853, 166)
clicking as dev xy: (1818, 72)


(1818, 72)